In [1]:
import subprocess as sp
import os
import pandas as pd
import math
import random


In [2]:
def clips_size_kB(path):
    try:
        return int(sp.run("du -s " + path, stdout=sp.PIPE, shell=True).stdout.decode("utf-8").split("\t")[0])
    except ValueError:
        pass
    return 0

def sort_age_gen_occ(df):
    age_gen_occ_dict = {}
    for age in dict(df.age.value_counts()):
        for gender in ["male", "female"]:
            age_gen_occ_dict[(age, gender)] = len(df[(df.age == age) & (df.gender == gender)])
    age_gen_occ_list = sorted(age_gen_occ_dict, key=age_gen_occ_dict.get)
    return age_gen_occ_dict, age_gen_occ_list

def print(p=True, *args, **kwargs):
    if type(p) == bool:
        if p:
            return __builtins__.print(*args, **kwargs)
        return
    return __builtins__.print(p, *args, **kwargs)


In [3]:
def downsample_language(path, new_size_kB, extension=".wav", p=False):
    path = '/'.join(path.split('/')) + '/'
    old_size_kB = clips_size_kB(path + "clips")
    df = pd.read_csv(path + "validated.tsv", sep="\t")
    if old_size_kB <= new_size_kB:
        print(p, "\nthe dataset cannot be downsampled to this size because it is not large enough")
        return [clip.split('.')[0] + '.' + extension.split('.')[-1] for clip in list(df.path)]
    
    df = pd.read_csv(path + "validated.tsv", sep="\t")
    clips_needed = int((new_size_kB / old_size_kB) * len(df))
    age_dict = dict(df.age.value_counts())
    clips_per_group = math.ceil(clips_needed / (len(age_dict) * 2))
    
    age_gen_occ_dict, age_gen_occ_list = sort_age_gen_occ(df)
    
    selected_clips = []
    i = 0
    print(p, "data length:", len(df), "\t\tclips needed:", clips_needed)
    for age, gender in age_gen_occ_list:

        i += 1
        current = df[(df.age == age) & (df.gender == gender)]
        now_selected = []
        if len(current) < clips_per_group:
            print(p, "\tclips_per_group was", clips_per_group)
            clips_needed -= len(current)
            try:
                clips_per_group = math.ceil(clips_needed / ((len(age_dict) * 2) - i))
            except ZeroDivisionError:
                clips_per_group = clips_needed
            now_selected = current
            selected_clips += list(now_selected.path)
            print(p, "\tnot enough clips for", age, gender, "so appending", len(now_selected))
            print(p, "\tclips_per_group now is", clips_per_group)
        else:
            now_selected = current.sample(clips_per_group)
            selected_clips += list(now_selected.path)
            clips_needed -= clips_per_group
            print(p, "\tenough clips for", age, gender, \
                  "(" + str(age_gen_occ_dict[(age, gender)]), "total), so appending", len(now_selected))
        df.drop(now_selected.index, inplace=True)
        print(p, "\ndata length:", len(df), "\t\tclips needed:", clips_needed)
            
    if clips_needed > 0:
        if len(df) < clips_needed:
            print(p, "\nnot enough labeled clips found, so appending", clips_needed, "unlabeled ones")
            selected_clips += list(df.sample(clips_needed).path)
        else:
            print(p, "\nthe dataset cannot be downsampled to this size because it is not large enough")
            selected_clips += list(df.path)
            
    return [clip.split('.')[0] + '.' + extension.split('.')[-1] for clip in selected_clips]
        
    
len(downsample_language("Data/nl", 36000))


821

In [4]:
def split_train_test_validate(data, weights):
    if type(weights) != dict:
        try:
            weights = {"train": weights[0], "test": weights[1], "validate": weights[2]}
        except:
            print("Geen iterable")
            return
        
    weights = {k:w/sum(weights.values()) for k, w in weights.items()}
    random.shuffle(data)
    
    train = data[:math.ceil(len(data) * weights["train"])]
    test = data[math.ceil(len(data) * weights["train"]):\
                math.ceil(len(data) * (weights["train"] + weights["test"]))]
    validate = data[math.ceil(len(data) * (weights["train"] + weights["test"])):]
    
    return {"train":train, "test":test, "validate":validate}
    
len(split_train_test_validate(downsample_language("Data/nl", 3000), [7, 2, 1])["train"])


54

In [5]:
def downsample_wrapped(path, new_size_kB, extension=".wav", p=False, destination_name="/general"):
    path = '/'.join(path.split('/')) + '/'
    destination = "Downsampled/" + path.split('/')[-2] + '/' + \
                '/'.join([c for c in destination_name.split('/') if c]) 
    
    sp.run("rm -r " + destination, shell=True)
    sp.run("mkdir Downsampled", shell=True)
    sp.run("mkdir " + '/'.join(destination.split('/')[:2]), shell=True)
    sp.run("mkdir " + destination, shell=True)

    samples = downsample_language(path, new_size_kB, extension, p)
    for sample in samples:
        sp.run("cp "+ path + "clips/" + sample + ' ' + destination, shell=True)
    
downsample_wrapped("Data/nl", 3000, extension=".mp3")


In [6]:
def downsample_split_wrapped(path, new_size_kB, split_weights, extension=".wav", p=False):    
    path = '/'.join(path.split('/')) + '/'
    
    sp.run("mkdir Downsampled", shell=True)
    sp.run("mkdir Downsampled/" + path.split('/')[-2], shell=True)

    samples = downsample_language(path, new_size_kB, extension, p)
    splits = split_train_test_validate(samples, split_weights)

    for split in splits:
        destination = "Downsampled/" + path.split('/')[-2] + "/" + split
        sp.run("rm -r " + destination, shell=True)
        sp.run("mkdir " + destination, shell=True)
        for sample in splits[split]:
            copy = sp.run("cp "+ path + "clips/" + sample + ' ' + destination, shell=True)
            print(p, "completed copying", sample, "to", destination, "with return code", copy.returncode)
            
downsample_split_wrapped("Data/nl", 3000, [7, 2, 1], extension=".mp3", p=True)


data length: 22954 		clips needed: 68
	clips_per_group was 6
	not enough clips for teens female so appending 0
	clips_per_group now is 7

data length: 22954 		clips needed: 68
	enough clips for fifties female (14 total), so appending 7

data length: 22947 		clips needed: 61
	enough clips for thirties female (15 total), so appending 7

data length: 22940 		clips needed: 54
	enough clips for sixties female (49 total), so appending 7

data length: 22933 		clips needed: 47
	enough clips for fourties female (50 total), so appending 7

data length: 22926 		clips needed: 40
	enough clips for sixties male (245 total), so appending 7

data length: 22919 		clips needed: 33
	enough clips for twenties female (302 total), so appending 7

data length: 22912 		clips needed: 26
	enough clips for teens male (1304 total), so appending 7

data length: 22905 		clips needed: 19
	enough clips for thirties male (2416 total), so appending 7

data length: 22898 		clips needed: 12
	enough clips for fourties mal